<a href="https://colab.research.google.com/github/izibili123/Data-Modeling-in-Data-Engineering/blob/main/Applied_Learning_Assignment_1_isaiah.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Step 1: Enter the Original Table

Use pandas DataFrame to represent the original table:

In [1]:
import pandas as pd
# Original Student Table (Unnormalized)
data = {
    "StudentID": [1, 2, 3],
    "StudentName": ["Amina Bello", "Tunde Julius", "Fatima Musa"],
    "Courses": ["Math, English", "English", "Math, Biology, Chemistry"],
    "Instructor": ["Adams, Smith", "Smith", "Adams, Grey, Obi"],
    "InstructorPhone": ["1234,5678", "5678", "1234,9101,1123"]
}

df = pd.DataFrame(data)
df

,StudentID,StudentName,Courses,Instructor,InstructorPhone
0,1,Amina Bello,"Math, English","Adams, Smith","1234,5678"
1,2,Tunde Julius,English,Smith,5678
2,3,Fatima Musa,"Math, Biology, Chemistry","Adams, Grey, Obi","1234,9101,1123"


Step 3: Convert to 1NF (First Normal Form)

Rule: Remove multi-valued attributes (Courses, Instructor, InstructorPhone).

Each record must hold only atomic values.

We split the rows so each student-course pair is separate:

In [4]:
# 2. Use the authoritative mapping you provided (Note)
course_to_instructor = {
    "Math": "Adams",
    "English": "Smith",
    "Biology": "Grey",
    "Chemistry": "Obi"
}

# If you also want a phone map (derived from dataset or provided)
instructor_to_phone = {
    "Adams": "1234",
    "Smith": "5678",
    "Grey": "9101",
    "Obi": "1123"
}

# 3. Build a proper 1NF table: one (atomic) course per row, with instructor & phone assigned by mapping
rows = []
for _, r in df.iterrows():
    sid = r["StudentID"]
    sname = r["StudentName"]
    # split courses into atomic values
    course_list = [c.strip() for c in r["Courses"].split(",")]
    for course in course_list:
        instr = course_to_instructor.get(course)            # map course -> instructor
        phone = instructor_to_phone.get(instr, "")          # map instructor -> phone (fallback empty)
        rows.append([sid, sname, course, instr, phone])

df_1NF = pd.DataFrame(rows, columns=["StudentID", "StudentName", "Course", "Instructor", "InstructorPhone"])

# 4. Show result
print("=== 1NF table (atomic values only) ===")
display(df_1NF)

=== 1NF table (atomic values only) ===


,StudentID,StudentName,Course,Instructor,InstructorPhone
0,1,Amina Bello,Math,Adams,1234
1,1,Amina Bello,English,Smith,5678
2,2,Tunde Julius,English,Smith,5678
3,3,Fatima Musa,Math,Adams,1234
4,3,Fatima Musa,Biology,Grey,9101
5,3,Fatima Musa,Chemistry,Obi,1123


Step 4 convert to 2NF:

Table must already be in 1NF (atomic values only).

If the primary key is composite, we must remove partial dependencies (non-key attributes depending on only part of the key).

In your case:

In the 1NF table, the natural key is (StudentID, Course) (since one student can take multiple courses).

StudentName depends only on StudentID (not on Course) → partial dependency.

InstructorPhone depends only on Instructor, not on (StudentID, Course) → another partial dependency.

👉 So we split into separate tables:

Student Table → StudentID, StudentName

Enrollment Table → StudentID, Course, Instructor

Instructor Table → Instructor, InstructorPhone

In [5]:
import pandas as pd

# Recreate 1NF table
rows = [
    ["001", "Amina Bello", "Math", "Adams", "1234"],
    ["001", "Amina Bello", "English", "Smith", "5678"],
    ["002", "Tunde Julius", "English", "Smith", "5678"],
    ["003", "Fatima Musa", "Math", "Adams", "1234"],
    ["003", "Fatima Musa", "Biology", "Grey", "9101"],
    ["003", "Fatima Musa", "Chemistry", "Obi", "1123"]
]
df_1NF = pd.DataFrame(rows, columns=["StudentID", "StudentName", "Course", "Instructor", "InstructorPhone"])

print("=== 1NF Table ===")
display(df_1NF)

# ---- 2NF Decomposition ----

# Student Table (remove partial dependency: StudentName depends only on StudentID)
student_table = df_1NF[["StudentID", "StudentName"]].drop_duplicates()

# Instructor Table (remove partial dependency: Phone depends only on Instructor)
instructor_table = df_1NF[["Instructor", "InstructorPhone"]].drop_duplicates()

# Enrollment Table (bridges Students and Courses with Instructor)
enrollment_table = df_1NF[["StudentID", "Course", "Instructor"]].drop_duplicates()

print("=== Student Table (2NF) ===")
display(student_table)

print("=== Instructor Table (2NF) ===")
display(instructor_table)

print("=== Enrollment Table (2NF) ===")
display(enrollment_table)

=== 1NF Table ===


,StudentID,StudentName,Course,Instructor,InstructorPhone
0,001,Amina Bello,Math,Adams,1234
1,001,Amina Bello,English,Smith,5678
2,002,Tunde Julius,English,Smith,5678
3,003,Fatima Musa,Math,Adams,1234
4,003,Fatima Musa,Biology,Grey,9101
5,003,Fatima Musa,Chemistry,Obi,1123


=== Student Table (2NF) ===


,StudentID,StudentName
0,001,Amina Bello
2,002,Tunde Julius
3,003,Fatima Musa


=== Instructor Table (2NF) ===


,Instructor,InstructorPhone
0,Adams,1234
1,Smith,5678
4,Grey,9101
5,Obi,1123


=== Enrollment Table (2NF) ===


,StudentID,Course,Instructor
0,001,Math,Adams
1,001,English,Smith
2,002,English,Smith
3,003,Math,Adams
4,003,Biology,Grey
5,003,Chemistry,Obi


Final 3NF Tables

Student Table (depends only on StudentID)

StudentID (PK)

StudentName

Instructor Table (depends only on Instructor)

Instructor (PK)

InstructorPhone

Course Table (depends only on Course)

Course (PK)

Instructor (FK → Instructor.Instructor)

Enrollment Table (bridges Students and Courses)

StudentID (FK → Student.StudentID)

Course (FK → Course.Course)

In [6]:
import pandas as pd

# Original 1NF rows
rows = [
    ["001", "Amina Bello", "Math", "Adams", "1234"],
    ["001", "Amina Bello", "English", "Smith", "5678"],
    ["002", "Tunde Julius", "English", "Smith", "5678"],
    ["003", "Fatima Musa", "Math", "Adams", "1234"],
    ["003", "Fatima Musa", "Biology", "Grey", "9101"],
    ["003", "Fatima Musa", "Chemistry", "Obi", "1123"]
]
df_1NF = pd.DataFrame(rows, columns=["StudentID", "StudentName", "Course", "Instructor", "InstructorPhone"])

# ---- 3NF Decomposition ----

# Student Table
student_table = df_1NF[["StudentID", "StudentName"]].drop_duplicates()

# Instructor Table
instructor_table = df_1NF[["Instructor", "InstructorPhone"]].drop_duplicates()

# Course Table (each course taught by one instructor)
course_table = df_1NF[["Course", "Instructor"]].drop_duplicates()

# Enrollment Table (relationship between Student and Course)
enrollment_table = df_1NF[["StudentID", "Course"]].drop_duplicates()

print("=== Student Table (3NF) ===")
display(student_table)

print("=== Instructor Table (3NF) ===")
display(instructor_table)

print("=== Course Table (3NF) ===")
display(course_table)

print("=== Enrollment Table (3NF) ===")
display(enrollment_table)


=== Student Table (3NF) ===


,StudentID,StudentName
0,001,Amina Bello
2,002,Tunde Julius
3,003,Fatima Musa


=== Instructor Table (3NF) ===


,Instructor,InstructorPhone
0,Adams,1234
1,Smith,5678
4,Grey,9101
5,Obi,1123


=== Course Table (3NF) ===


,Course,Instructor
0,Math,Adams
1,English,Smith
4,Biology,Grey
5,Chemistry,Obi


=== Enrollment Table (3NF) ===


,StudentID,Course
0,001,Math
1,001,English
2,002,English
3,003,Math
4,003,Biology
5,003,Chemistry
